In [1]:
import warnings, json, numpy as np, pandas as pd, dash, plotly.express as px, plotly.graph_objects as go
from textwrap import dedent as d
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

STOPWORDS = {'a', 'actualmente', 'adelante', 'además', 'afirmó', 'agregó', 'ahora', 'ahí', 'al', 'algo', 'alguna', 'algunas', 'alguno', 'algunos', 'algún', 'alrededor', 'ambos', 'ampleamos', 'ante', 'anterior', 'antes', 'apenas', 'aproximadamente', 'aquel', 'aquellas', 'aquellos', 'aqui', 'aquí', 'arriba', 'aseguró', 'así', 'atras', 'aunque', 'ayer', 'añadió', 'aún', 'bajo', 'bastante', 'bien', 'buen', 'buena', 'buenas', 'bueno', 'buenos', 'cada', 'casi', 'cerca', 'cierta', 'ciertas', 'cierto', 'ciertos', 'cinco', 'comentó', 'como', 'co', 'con', 'conocer', 'conseguimos', 'conseguir', 'considera', 'consideró', 'consigo', 'consigue', 'consiguen', 'consigues', 'contra', 'cosas', 'creo', 'cual', 'cuales', 'cualquier', 'cuando', 'cuanto', 'cuatro', 'cuenta', 'cómo', 'da', 'dado', 'dan', 'dar', 'de', 'debe', 'deben', 'debido', 'decir', 'dejó', 'del', 'demás', 'dentro', 'desde', 'después', 'dice', 'dicen', 'dicho', 'dieron', 'diferente', 'diferentes', 'dijeron', 'dijo', 'dio', 'donde', 'dos', 'durante', 'e', 'ejemplo', 'el', 'él', 'ella', 'ellas', 'ello', 'ellos', 'embargo', 'empleais', 'emplean', 'emplear', 'empleas', 'empleo', 'en', 'encima', 'encuentra', 'entonces', 'entre', 'era', 'erais', 'eramos', 'eran', 'eras', 'eres', 'es', 'esa', 'esas', 'ese', 'eso', 'esos', 'esta', 'estaba', 'estabais', 'estaban', 'estabas', 'estad', 'estada', 'estadas', 'estado', 'estados', 'estais', 'estamos', 'estan', 'estando', 'estar', 'estaremos', 'estará', 'estarán', 'estarás', 'estaré', 'estaréis', 'estaría', 'estaríais', 'estaríamos', 'estarían', 'estarías', 'estas', 'este', 'estemos', 'esto', 'estos', 'estoy', 'estuve', 'estuviera', 'estuvierais', 'estuvieran', 'estuvieras', 'estuvieron', 'estuviese', 'estuvieseis', 'estuviesen', 'estuvieses', 'estuvimos', 'estuviste', 'estuvisteis', 'estuviéramos', 'estuviésemos', 'estuvo', 'está', 'estábamos', 'estáis', 'están', 'estás', 'esté', 'estéis', 'estén', 'estés', 'ex', 'existe', 'existen', 'explicó', 'expresó', 'fin', 'fue', 'fuera', 'fuerais', 'fueran', 'fueras', 'fueron', 'fuese', 'fueseis', 'fuesen', 'fueses', 'fui', 'fuimos', 'fuiste', 'fuisteis', 'fuéramos', 'fuésemos', 'gran', 'grandes', 'gueno', 'ha', 'haber', 'habida', 'habidas', 'habido', 'habidos', 'habiendo', 'habremos', 'habrá', 'habrán', 'habrás', 'habré', 'habréis', 'habría', 'habríais', 'habríamos', 'habrían', 'habrías', 'habéis', 'había', 'habíais', 'habíamos', 'habían', 'habías', 'hace', 'haceis', 'hacemos', 'hacen', 'hacer', 'hacerlo', 'haces', 'hacia', 'haciendo', 'hago', 'han', 'has', 'hasta', 'hay', 'haya', 'hayamos', 'hayan', 'hayas', 'hayáis', 'he', 'hecho', 'hemos', 'hicieron', 'hizo', 'hoy', 'https', 'hube', 'hubiera', 'hubierais', 'hubieran', 'hubieras', 'hubieron', 'hubiese', 'hubieseis', 'hubiesen', 'hubieses', 'hubimos', 'hubiste', 'hubisteis', 'hubiéramos', 'hubiésemos', 'hubo', 'igual', 'incluso', 'indicó', 'informó', 'intenta', 'intentais', 'intentamos', 'intentan', 'intentar', 'intentas', 'intento', 'ir', 'junto', 'la', 'lado', 'largo', 'las', 'le', 'les', 'llegó', 'lleva', 'llevar', 'lo', 'los', 'luego', 'lugar', 'manera', 'manifestó', 'mayor', 'me', 'mediante', 'mejor', 'mencionó', 'menos', 'mi', 'mientras', 'mio', 'mis', 'misma', 'mismas', 'mismo', 'mismos', 'modo', 'momento', 'mucha', 'muchas', 'mucho', 'muchos', 'muy', 'más', 'mí', 'mía', 'mías', 'mío', 'míos', 'nada', 'nadie', 'ni', 'ninguna', 'ningunas', 'ninguno', 'ningunos', 'ningún', 'no', 'nos', 'nosotras', 'nosotros', 'nuestra', 'nuestras', 'nuestro', 'nuestros', 'nueva', 'nuevas', 'nuevo', 'nuevos', 'nunca', 'o', 'ocho', 'os', 'otra', 'otras', 'otro', 'otros', 'para', 'parece', 'parte', 'partir', 'pasada', 'pasado', 'pero', 'pesar', 'poca', 'pocas', 'poco', 'pocos', 'podeis', 'podemos', 'poder', 'podria', 'podriais', 'podriamos', 'podrian', 'podrias', 'podrá', 'podrán', 'podría', 'podrían', 'poner', 'por', 'por qué', 'porque', 'posible', 'primer', 'primera', 'primero', 'primeros', 'principalmente', 'propia', 'propias', 'propio', 'propios', 'próximo', 'próximos', 'pudo', 'pueda', 'puede', 'pueden', 'puedo', 'pues', 'que', 'quedó', 'queremos', 'quien', 'quienes', 'quiere', 'quién', 'qué', 'realizado', 'realizar', 'realizó', 'respecto', 'sabe', 'sabeis', 'sabemos', 'saben', 'saber', 'sabes', 'se', 'sea', 'seamos', 'sean', 'seas', 'segunda', 'segundo', 'según', 'seis', 'ser', 'seremos', 'será', 'serán', 'serás', 'seré', 'seréis', 'sería', 'seríais', 'seríamos', 'serían', 'serías', 'seáis', 'señaló', 'si', 'sido', 'siempre', 'siendo', 'siete', 'sigue', 'siguiente', 'sin', 'sino', 'sobre', 'sois', 'sola', 'solamente', 'solas', 'solo', 'solos', 'somos', 'son', 'soy', 'su', 'sus', 'suya', 'suyas', 'suyo', 'suyos', 'sí', 'sólo', 'tal', 'también', 'tampoco', 'tan', 'tanto', 'te', 'tendremos', 'tendrá', 'tendrán', 'tendrás', 'tendré', 'tendréis', 'tendría', 'tendríais', 'tendríamos', 'tendrían', 'tendrías', 'tened', 'teneis', 'tenemos', 'tener', 'tenga', 'tengamos', 'tengan', 'tengas', 'tengo', 'tengáis', 'tenida', 'tenidas', 'tenido', 'tenidos', 'teniendo', 'tenéis', 'tenía', 'teníais', 'teníamos', 'tenían', 'tenías', 'tercera', 'ti', 'tiempo', 'tiene', 'tienen', 'tienes', 'toda', 'todas', 'todavía', 'todo', 'todos', 'total', 'trabaja', 'trabajais', 'trabajamos', 'trabajan', 'trabajar', 'trabajas', 'trabajo', 'tras', 'trata', 'través', 'tres', 'tu', 'tus', 'tuve', 'tuviera', 'tuvierais', 'tuvieran', 'tuvieras', 'tuvieron', 'tuviese', 'tuvieseis', 'tuviesen', 'tuvieses', 'tuvimos', 'tuviste', 'tuvisteis', 'tuviéramos', 'tuviésemos', 'tuvo', 'tuya', 'tuyas', 'tuyo', 'tuyos', 'tú', 'ultimo', 'un', 'una', 'unas', 'uno', 'unos', 'usa', 'usais', 'usamos', 'usan', 'usar', 'usas', 'uso', 'usted', 'va', 'vais', 'valor', 'vamos', 'van', 'varias', 'varios', 'vaya', 'veces', 'ver', 'verdad', 'verdadera', 'verdadero', 'vez', 'vosotras', 'vosotros', 'voy', 'vuestra', 'vuestras', 'vuestro', 'vuestros', 'y', 'ya', 'yo', 'él', 'éramos', 'ésta', 'éstas', 'éste', 'éstos', 'última', 'últimas', 'último', 'últimos'}

import pyLDAvis
import pyLDAvis.sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import string, re, sklearn, pandas as pd, numpy as np
#from wordcloud import WordCloud
from subprocess import check_output
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

ruta = 'C://Tweetsentpol.csv'
df = pd.read_csv(ruta, header=0)
df2 = df[['Fecha', 'Polarity']]

fecha_ini = '2021-08-01'
fecha_fin = '2021-12-31'
mask = (df2['Fecha'] >= fecha_ini) & (df2['Fecha'] <= fecha_fin)
df3 = df2.loc[mask]
df3 = df3.groupby(['Fecha']).mean()
df4 = df3.reset_index()
df4['Color'] = "Fecha"

warnings.simplefilter(action='ignore', category=FutureWarning)

app = JupyterDash(__name__)
styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}

fig = px.line(df4, x="Fecha", y="Polarity", color="Color", title="No label selected")
fig.update_traces(mode="markers+lines")

app.layout = html.Div([
    dcc.Graph(
        id='figure1',
        figure=fig,
    ),
    
    html.Div(className='row', children=[
        html.Div([
            dcc.Markdown(d("""Hoverdata using figure references""")),
            html.Pre(id='hoverdata2', style=styles['pre']),
        ], className='three columns'),
    ])
])

store = []

ruta2 = 'C://Tweets.csv'
dftuits = pd.read_csv(ruta2, header=0)
stopwords = set(STOPWORDS)

def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words=stopwords).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

vectorizer = CountVectorizer(analyzer='word', min_df=3, stop_words=stopwords, lowercase=True, 
                             token_pattern='[a-zA-Z0-9]{3,}', max_features=5000)

lda_model = LatentDirichletAllocation(n_components=5, learning_method='online', random_state=20, n_jobs = -1)

In [2]:
@app.callback(
    Output('hoverdata2', 'children'),
    [Input('figure1', 'clickData')])

def display_hover_data(hoverData):
    if hoverData is not None:
        traceref = hoverData['points'][0]['curveNumber']
        pointref = hoverData['points'][0]['pointNumber']

        mask2 = (dftuits['fecha'] == fig.data[traceref]['x'][pointref])
        dftuits2 = dftuits.loc[mask2]
        lostuits = pd.Series(dftuits2['tweet'])
        lostuits2 = lostuits.astype(str).str.cat(sep='\n')
        
        common_words = get_top_n_words(dftuits2.tweet, 10)
        b = [i[0] for i in common_words]
        toptenwords = "Los términos mas frecuentes en este periodo son: " + ", ".join(b)
#LDA
        data_matrix = vectorizer.fit_transform(dftuits2.tweet)
        lda_output = lda_model.fit_transform(data_matrix)
        cadena = ""
        for i,topic in enumerate(lda_model.components_):
            subcadena = [vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]]
            palabras = ', '.join([str(item) for item in subcadena])
            cadena = cadena + 'Top 10 palabras para el tópico #' + str(i) + ": " + palabras + '\n'
        cadena = cadena + '\n' + ">>> Esta es la muestra de tweets recolectados del día <<<"
        
        return html.Div([
                html.Pre('Dia Seleccionado ' + fig.data[traceref]['x'][pointref]),html.Pre(toptenwords),html.Pre(cadena),html.Pre(lostuits2)
            ])
    else:
        return 'None selected'
    
app.run_server(mode='external', port = 7078, dev_tools_ui=True,
          dev_tools_hot_reload =True, threaded=True)

C:\Users\jose.jimenez\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



Dash app running on http://127.0.0.1:7078/


C:\Users\jose.jimenez\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['adem', 'adi', 'afirm', 'agreg', 'ais', 'alg', 'amos', 'aqu', 'asegur', 'bamos', 'coment', 'consider', 'dej', 'dem', 'despu', 'est', 'estuvi', 'explic', 'expres', 'hab', 'habr', 'hubi', 'indic', 'inform', 'lleg', 'ltima', 'ltimas', 'ltimo', 'ltimos', 'manifest', 'mencion', 'ning', 'podr', 'qued', 'qui', 'ramos', 'realiz', 'seg', 'semos', 'sta', 'stas', 'ste', 'stos', 'tambi', 'ten', 'tendr', 'teng', 'todav', 'trav', 'tuvi', 'ximo', 'ximos'] not in stop_words.

